In [ ]:
import requests
import zipfile

### Download and Extract BIL Files

In [ ]:
response = requests.get(r'https://www.prism.oregonstate.edu/fetchData.php?type=all_bil&kind=normals&spatial=4km&elem=ppt&temporal=annual')
bilpath = r"C:\Users\gregkohler1\GIS\Lab2\Lab2\30yearppt.zip"
if response.status_code == 200:
    with open(bilpath, 'wb') as file:
        file.write(response.content)
        
with zipfile.ZipFile(bilpath, 'r') as zip_ref:
        # Extract all contents to the extraction folder
        zip_ref.extractall(r"C:\Users\gregkohler1\GIS\Lab2\Lab2\BIL")

#### Convert Bil to Usable Format

In [ ]:
### BIL TO TIFF
arcpy.conversion.RasterToOtherFormat(
    Input_Rasters=r"C:\Users\gregkohler1\GIS\Lab2\Lab2\BIL\PRISM_ppt_30yr_normal_4kmM4_01_bil.bil;C:\Users\gregkohler1\GIS\Lab2\Lab2\BIL\PRISM_ppt_30yr_normal_4kmM4_02_bil.bil;C:\Users\gregkohler1\GIS\Lab2\Lab2\BIL\PRISM_ppt_30yr_normal_4kmM4_03_bil.bil;C:\Users\gregkohler1\GIS\Lab2\Lab2\BIL\PRISM_ppt_30yr_normal_4kmM4_04_bil.bil;C:\Users\gregkohler1\GIS\Lab2\Lab2\BIL\PRISM_ppt_30yr_normal_4kmM4_05_bil.bil;C:\Users\gregkohler1\GIS\Lab2\Lab2\BIL\PRISM_ppt_30yr_normal_4kmM4_06_bil.bil;C:\Users\gregkohler1\GIS\Lab2\Lab2\BIL\PRISM_ppt_30yr_normal_4kmM4_07_bil.bil;C:\Users\gregkohler1\GIS\Lab2\Lab2\BIL\PRISM_ppt_30yr_normal_4kmM4_08_bil.bil;C:\Users\gregkohler1\GIS\Lab2\Lab2\BIL\PRISM_ppt_30yr_normal_4kmM4_09_bil.bil;C:\Users\gregkohler1\GIS\Lab2\Lab2\BIL\PRISM_ppt_30yr_normal_4kmM4_10_bil.bil;C:\Users\gregkohler1\GIS\Lab2\Lab2\BIL\PRISM_ppt_30yr_normal_4kmM4_11_bil.bil;C:\Users\gregkohler1\GIS\Lab2\Lab2\BIL\PRISM_ppt_30yr_normal_4kmM4_12_bil.bil;C:\Users\gregkohler1\GIS\Lab2\Lab2\BIL\PRISM_ppt_30yr_normal_4kmM4_annual_bil.bil",
    Output_Workspace=r"C:\Users\gregkohler1\GIS\Lab2\Lab2\BILTOTIF",
    Raster_Format="TIFF"
)

#### Create Mosaic Dataset

In [ ]:
with arcpy.EnvManager(outputCoordinateSystem='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]'):
    arcpy.management.CreateMosaicDataset(
        in_workspace=r"C:\Users\gregkohler1\GIS\Lab2\Lab2\Lab2Data.gdb",
        in_mosaicdataset_name="PRISM",
        coordinate_system='GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
        num_bands=None,
        pixel_type="",
        product_definition="NONE",
        product_band_definitions=None
    )

#### Add Raster Files to Mosaic

In [ ]:
arcpy.management.AddRastersToMosaicDataset(
    in_mosaic_dataset="PRISM",
    raster_type="Raster Dataset",
    input_path=r"C:\Users\gregkohler1\GIS\Lab2\Lab2\BILTOTIF",
    update_cellsize_ranges="UPDATE_CELL_SIZES",
    update_boundary="UPDATE_BOUNDARY",
    update_overviews="NO_OVERVIEWS",
    maximum_pyramid_levels=None,
    maximum_cell_size=0,
    minimum_dimension=1500,
    spatial_reference=None,
    filter="",
    sub_folder="SUBFOLDERS",
    duplicate_items_action="ALLOW_DUPLICATES",
    build_pyramids="NO_PYRAMIDS",
    calculate_statistics="NO_STATISTICS",
    build_thumbnails="NO_THUMBNAILS",
    operation_description="",
    force_spatial_reference="NO_FORCE_SPATIAL_REFERENCE",
    estimate_statistics="NO_STATISTICS",
    aux_inputs=None,
    enable_pixel_cache="NO_PIXEL_CACHE",
    cache_location=r"C:\Users\gregkohler1\AppData\Local\ESRI\rasterproxies\PRISM"
)

#### Calculate Variable Field

In [ ]:
arcpy.management.CalculateField(
    in_table=r"PRISM\Footprint",
    field="Variable",
    expression='"PRISM"',
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

#### Calculate Field for Time

In [ ]:
arcpy.management.CalculateField(
    in_table=r"PRISM\Footprint",
    field="Timestamp",
    expression="""var month = $feature.OBJECTID;
var year = 1991;
if (month == 13) {
   var dateValue = null;
} else {
   var adjustedMonth = month - 1; 
   var dateValue = Date(year, adjustedMonth, 1);
}
dateValue
""",
    expression_type="ARCADE",
    code_block="",
    field_type="DATE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

####  Build Multidimensional Info

In [ ]:
arcpy.md.BuildMultidimensionalInfo(
    in_mosaic_dataset="PRISM",
    variable_field="Variable",
    dimension_fields="Timestamp # #",
    variable_desc_units=None,
    delete_multidimensional_info="NO_DELETE_MULTIDIMENSIONAL_INFO"
)

#### Make Multidimensional Raster Layer

In [ ]:
arcpy.md.MakeMultidimensionalRasterLayer(
    in_multidimensional_raster="PRISM",
    out_multidimensional_raster_layer="PRISM_MultidimLayer",
    variables="PRISM",
    dimension_def="ALL",
    dimension_ranges=None,
    dimension_values=None,
    dimension="",
    start_of_first_iteration="",
    end_of_first_iteration="",
    iteration_step=None,
    iteration_unit="",
    template='-125.020833333333 24.0624999997935 -66.4791666661985 49.9375000000005 GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
    dimensionless="DIMENSIONS",
    spatial_reference=None
)

#### Create Space Time Cubes

In [ ]:
arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer(
    in_md_raster="PRISM_MultidimLayer",
    output_cube=r"C:\Users\gregkohler1\GIS\Lab2\Lab2\PRISM_CUBES.nc",
    fill_empty_bins="ZEROS"
)